In [1]:
#Import Libraries

from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

In [2]:
num_classes = 5
img_rows,img_cols = 48,48
batch_size = 32

In [3]:
#Load the data
train_data_dir='C:/Users/21653/Desktop/face-expression-recognition-dataset/face-expression-recognition-dataset/train'
validation_data_dir='C:/Users/21653/Desktop/face-expression-recognition-dataset/face-expression-recognition-dataset/validation'

In [4]:
#Augmentation data
train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					width_shift_range=0.4,
					height_shift_range=0.4,
					horizontal_flip=True,
					fill_mode='nearest')

In [5]:
validation_datagen=ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)





Found 24282 images belonging to 5 classes.


In [7]:
validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(img_rows,img_cols),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)

Found 5937 images belonging to 5 classes.


In [8]:
len(validation_generator)

186

In [9]:
len(train_generator)

759

In [10]:
train_generator.dtype

'float32'

In [11]:

# BUILDING MODEL

model = Sequential()

# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [12]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('Emotion_little_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])





In [13]:
#Model training
nb_train_samples = 24172
nb_validation_samples = 3006
epochs=25

history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
755/755 [==============================] - ETA: 0s - loss: 1.8388 - accuracy: 0.2393
Epoch 00001: val_loss improved from inf to 1.54811, saving model to Emotion_little_vgg.h5
755/755 [==============================] - 728s 964ms/step - loss: 1.8388 - accuracy: 0.2393 - val_loss: 1.5481 - val_accuracy: 0.3054
Epoch 2/25
755/755 [==============================] - ETA: 0s - loss: 1.5747 - accuracy: 0.2824
Epoch 00002: val_loss improved from 1.54811 to 1.52318, saving model to Emotion_little_vgg.h5
755/755 [==============================] - 661s 876ms/step - loss: 1.5747 - accuracy: 0.2824 - val_loss: 1.5232 - val_accuracy: 0.3256
Epoch 3/25
755/755 [==============================] - ETA: 0s - loss: 1.5523 - accuracy: 0.2964
Epoch 00003: val_loss improved from 1.52318 to 1.49994, saving model to Emotion_little_vgg.h5
755/755 [==============================] - 661s 875ms/step - loss: 1.5523 - accuracy: 0.

In [28]:

model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=10,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)
import cv2 

Epoch 1/10
755/755 [==============================] - ETA: 0s - loss: 1.0727 - accuracy: 0.5759
Epoch 00001: val_loss improved from 0.84435 to 0.80980, saving model to Emotion_little_vgg.h5
755/755 [==============================] - 867s 1s/step - loss: 1.0727 - accuracy: 0.5759 - val_loss: 0.8098 - val_accuracy: 0.6747
Epoch 2/10
755/755 [==============================] - ETA: 0s - loss: 1.0526 - accuracy: 0.5818
Epoch 00002: val_loss did not improve from 0.80980
755/755 [==============================] - 861s 1s/step - loss: 1.0526 - accuracy: 0.5818 - val_loss: 0.8115 - val_accuracy: 0.6794
Epoch 3/10
755/755 [==============================] - ETA: 0s - loss: 1.0522 - accuracy: 0.5823
Epoch 00003: val_loss improved from 0.80980 to 0.79518, saving model to Emotion_little_vgg.h5
755/755 [==============================] - 867s 1s/step - loss: 1.0522 - accuracy: 0.5823 - val_loss: 0.7952 - val_accuracy: 0.6895
Epoch 4/10
755/755 [==============================] - ETA: 0s - loss: 1.0354 

In [ ]:
#saving our model
model.save('Facial-Expressions-Recognition-master2')

In [19]:
#Face detection
face_classifier=cv2.CascadeClassifier('C:/Users/21653/Desktop/Facial-Expressions-Recognition-master/Facial-Expressions-Recognition-master/haarcascade_frontalface_default.xml')

In [33]:
#Create a list of labels 
class_labels = ['Angry','Happy','Neutral','Sad','Surprise']
cap = cv2.VideoCapture(0)


In [34]:
model.save('Facial-Expressions-Recognition-master2')

INFO:tensorflow:Assets written to: Facial-Expressions-Recognition-master2\assets


In [22]:
import matplotlib.pyplot as plt

In [ ]:
len(faces)

In [35]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [ ]:

while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
    # rect,face,image = face_detector(frame)


        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

        # make a prediction on the ROI, then lookup the class

            preds = model.predict(roi)[0]
            label=class_labels[preds.argmax()]
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        else:
            cv2.putText(frame,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
